<!-- File automatically generated using DocOnce (https://github.com/doconce/doconce/):
doconce format ipynb interactive_gstar_function_v2.do.txt --encoding=utf-8 --ipynb_admon=hrule --ipynb_disable_mpl_inline --ipynb_cite=latex-plain -->

# Sensitivity indices for Sobol's $G^{*}$ function

**Leif Rune Hellevik**

# Setup

In [1]:
# --- cell: install_chaospy ---
# @title Install chaospy (Colab-friendly)

try:
    import chaospy as cp
    import numpoly
    import numpy as np
    print("chaospy er allerede installert.")
except ImportError:
    # Installer chaospy fra PyPI. Dette drar inn numpoly automatisk.
    %pip install chaospy==4.3.21 --no-cache-dir
    import chaospy as cp
    import numpoly
    import numpy as np

print("numpy  :", np.__version__)
print("numpoly:", numpoly.__version__)
print("chaospy:", cp.__version__)

In [2]:
# --- cell: repo_setup ---
# @title Repo sync and environment setup

import os
import sys
import subprocess
from pathlib import Path

IN_COLAB = "google.colab" in sys.modules
REMOTE = "https://github.com/lrhgit/uqsa2025.git"
REPO_PATH_COLAB = Path("/content/uqsa2025")

if IN_COLAB:
    if not REPO_PATH_COLAB.exists():
        print("Cloning repository...")
        subprocess.run(
            ["git", "clone", REMOTE, str(REPO_PATH_COLAB)],
            check=True
        )
    else:
        print("Updating existing repository...")
        subprocess.run(
            ["git", "-C", str(REPO_PATH_COLAB), "pull"],
            check=True
        )
    os.chdir(REPO_PATH_COLAB)

# --- Find repo root (works locally + in Colab) ---
cwd = Path.cwd().resolve()
repo_root = next(
    (p for p in [cwd] + list(cwd.parents) if (p / ".git").exists()),
    cwd
)

PY_SRC = repo_root / "python_source"
if PY_SRC.exists() and str(PY_SRC) not in sys.path:
    sys.path.insert(0, str(PY_SRC))

print("CWD:", Path.cwd())
print("repo_root:", repo_root)
print("python_source exists:", PY_SRC.exists())
print("python_source in sys.path:", str(PY_SRC) in sys.path)

In [3]:
# --- cell: layout_and_numpy_patch ---
# @title Layout fix, imports, and NumPy compatibility patch

import warnings
warnings.filterwarnings("ignore")

from IPython.display import HTML

HTML("""
<style>
div.cell.code_cell, div.output {
    max-width: 100% !important;
}
</style>
""")

import numpy as np
import matplotlib.pyplot as plt
import chaospy as cp
import numpoly
import pandas as pd

# --- NumPy reshape compatibility patch for numpoly ---
_old_reshape = np.reshape

def _reshape_compat(a, *args, **kwargs):
    newshape = None
    if "newshape" in kwargs:
        newshape = kwargs.pop("newshape")
    if "shape" in kwargs and newshape is None:
        newshape = kwargs.pop("shape")
    if newshape is not None:
        return _old_reshape(a, newshape, *args, **kwargs)
    return _old_reshape(a, *args, **kwargs)

np.reshape = _reshape_compat
print("✓ numpy.reshape patched for numpoly compatibility")

# Background

This notebook assumes familiarity with basic concepts in uncertainty propagation
and surrogate modelling, in particular

* Monte Carlo methods (MC)

* Polynomial Chaos Expansion (PCE)

These methods are introduced and discussed in detail in the notebook
`sensitivity_introduction.ipynb`.

The examples and results presented here reuse the concepts and notation
established in that introduction.

# Sobol's $G^{*}$ function

In [4]:
# --- cell: gstar_statistics ---

# Gstar-statistics
# import modules

import numpy as np


def Vi(ai, alphai):
    return alphai**2 / ((1 + 2 * alphai) * (1 + ai) ** 2)


def V(a_prms, alpha):
    D = 1.0
    for ai, alphai in zip(a_prms, alpha):
        D *= (1.0 + Vi(ai, alphai))
    return D - 1.0


def S_i(a, alpha):
    S_i = np.zeros_like(a)
    Vtot = V(a, alpha)
    for i, (ai, alphai) in enumerate(zip(a, alpha)):
        S_i[i] = Vi(ai, alphai) / Vtot
    return S_i


def S_T(a, alpha):
    S_T = np.zeros_like(a)
    Vtot = V(a, alpha)
    for i, (ai, alphai) in enumerate(zip(a, alpha)):
        S_T[i] = (Vtot + 1.0) / (Vi(ai, alphai) + 1.0) * Vi(ai, alphai) / Vtot
    return S_T

**Note on the parameter $\delta$**

The parameters $\delta_i$ are included only to keep the slider interface consistent with other Sobol test-function notebooks.
For Sobol’s $G^*$ function, the sensitivity indices depend only on $(a_i,\alpha_i)$, and $\delta_i$ does **not** enter the analytical expressions for $S_i$ or $S_i^T$.

In [5]:
# --- cell: gstar_sliders_colab ---

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

plt.rcParams["figure.figsize"] = (7, 4)

out = widgets.Output()

def mk_sliders(prefix, n, v0, vmin, vmax, step, desc=None):
    if desc is None:
        desc = [f"{prefix}{i}" for i in range(1, n+1)]
    return [
        widgets.FloatSlider(
            value=v0[i-1] if isinstance(v0, (list, tuple)) else v0,
            min=vmin, max=vmax, step=step,
            description=desc[i-1],
            continuous_update=False
        )
        for i in range(1, n+1)
    ]

# Parameters used in G*
a = mk_sliders("a", 4, [0.75, 0.80, 0.20, 0.20], 0.0, 2.0, 0.05)
alpha = mk_sliders("alpha", 4, [0.75, 0.20, 0.20, 0.20],
                   0.0, 1.0, 0.05,
                   desc=["α1", "α2", "α3", "α4"])

# delta sliders kept for UI consistency (not used in G*)
delta = mk_sliders("delta", 4, [0.60, 0.50, 0.20, 0.20],
                   0.0, 1.0, 0.05,
                   desc=["δ1", "δ2", "δ3", "δ4"])

def redraw(*_):
    with out:
        clear_output(wait=True)

        aval = [s.value for s in a]
        alphaval = [s.value for s in alpha]

        Si = S_i(aval, alphaval)
        ST = S_T(aval, alphaval)

        fig, ax = plt.subplots()
        x = np.arange(1, len(Si) + 1)

        ax.bar(x - 0.15, Si, width=0.3, label="Sᵢ")
        ax.bar(x + 0.15, ST, width=0.3, label="Sᵀ")

        ax.set_xticks(x)
        ax.set_xticklabels([f"X{i}" for i in x])
        ax.set_ylim(0, 1)
        ax.set_ylabel("Sensitivity index")
        ax.set_title("Sobol G* – sensitivities")
        ax.legend()

        plt.show()

for s in (*a, *alpha, *delta):
    s.observe(redraw, names="value")

ui = widgets.VBox([
    widgets.HBox(a),
    widgets.HBox(alpha),
    widgets.HBox(delta),
    out
])

display(ui)
redraw()